# Run some constrained projections

Follow the same setup as in notebook 200, with more scrutiny of outputs including other forcers.

We need to save out the land use forcing in each SSP and ensemble member to feed into the concentration-driven runs (240).

## Basic imports

In [ ]:
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
from tqdm import tqdm

from fair21 import (
    SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, CH4LifetimeMethod, FAIR
)
from fair21.defaults import species_config_from_default

## Set up problem

In [ ]:
species_ids = {
    # Greenhouse gases and precursors
    'CO2_FFI': SpeciesID('CO2 fossil fuel and industrial', Category.CO2_FFI),
    'CO2_AFOLU': SpeciesID('CO2 AFOLU', Category.CO2_AFOLU),
    'CO2': SpeciesID('CO2', Category.CO2),
    'CH4': SpeciesID('CH4', Category.CH4),
    'N2O': SpeciesID('N2O', Category.N2O),
    'CFC-11': SpeciesID('CFC-11', Category.CFC_11),
    'CFC-12': SpeciesID('CFC-12', Category.OTHER_HALOGEN),
    'CFC-113': SpeciesID('CFC-113', Category.OTHER_HALOGEN),
    'CFC-114': SpeciesID('CFC-114', Category.OTHER_HALOGEN),
    'CFC-115': SpeciesID('CFC-115', Category.OTHER_HALOGEN),
    'HCFC-22': SpeciesID('HCFC-22', Category.OTHER_HALOGEN),
    'HCFC-141b': SpeciesID('HCFC-141b', Category.OTHER_HALOGEN),
    'HCFC-142b': SpeciesID('HCFC-142b', Category.OTHER_HALOGEN),
    'CCl4': SpeciesID('CCl4', Category.OTHER_HALOGEN),
    'CHCl3': SpeciesID('CHCl3', Category.OTHER_HALOGEN),
    'CH2Cl2': SpeciesID('CH2Cl2', Category.OTHER_HALOGEN),
    'CH3Cl': SpeciesID('CH3Cl', Category.OTHER_HALOGEN),
    'CH3CCl3': SpeciesID('CH3CCl3', Category.OTHER_HALOGEN),
    'CH3Br': SpeciesID('CH3Br', Category.OTHER_HALOGEN),
    'Halon-1211': SpeciesID('Halon-1211', Category.OTHER_HALOGEN),
    'Halon-1301': SpeciesID('Halon-1301', Category.OTHER_HALOGEN),
    'Halon-2402': SpeciesID('Halon-2402', Category.OTHER_HALOGEN),
    'CF4': SpeciesID('CF4', Category.F_GAS),
    'C2F6': SpeciesID('C2F6', Category.F_GAS),
    'C3F8': SpeciesID('C3F8', Category.F_GAS),
    'c-C4F8': SpeciesID('C-C4F8', Category.F_GAS),
    'C4F10': SpeciesID('C4F10', Category.F_GAS),
    'C5F12': SpeciesID('C5F12', Category.F_GAS),
    'C6F14': SpeciesID('C6F14', Category.F_GAS),
    'C7F16': SpeciesID('C7F16', Category.F_GAS),
    'C8F18': SpeciesID('C8F18', Category.F_GAS),
    'HFC-125': SpeciesID('HFC-125', Category.F_GAS),
    'HFC-134a': SpeciesID('HFC-134a', Category.F_GAS),
    'HFC-143a': SpeciesID('HFC-143a', Category.F_GAS),
    'HFC-152a': SpeciesID('HFC-152a', Category.F_GAS),
    'HFC-227ea': SpeciesID('HFC-227ea', Category.F_GAS),
    'HFC-23': SpeciesID('HFC-23', Category.F_GAS),
    'HFC-236fa': SpeciesID('HFC-236fa', Category.F_GAS),
    'HFC-245fa': SpeciesID('HFC-245fa', Category.F_GAS),
    'HFC-32': SpeciesID('HFC-32', Category.F_GAS),
    'HFC-365mfc': SpeciesID('HFC-365mfc', Category.F_GAS),
    'HFC-4310mee': SpeciesID('HFC-4310mee', Category.F_GAS),
    'NF3': SpeciesID('NF3', Category.F_GAS),
    'SF6': SpeciesID('SF6', Category.F_GAS),
    'SO2F2': SpeciesID('SO2F2', Category.F_GAS),
    # aerosols, ozone, and their precursors
    'Sulfur': SpeciesID('Sulfur', Category.SULFUR),
    'BC': SpeciesID('BC', Category.BC),
    'OC': SpeciesID('OC', Category.OC),
    'NH3': SpeciesID('NH3', Category.OTHER_AEROSOL),
    'VOC': SpeciesID('VOC', Category.REACTIVE_GAS),
    'CO': SpeciesID('CO', Category.REACTIVE_GAS),
    'NOx': SpeciesID('NOx', Category.REACTIVE_GAS),
    'ari': SpeciesID('Aerosol-Radiation Interactions', Category.AEROSOL_RADIATION_INTERACTIONS),
    'aci': SpeciesID('Aerosol-Cloud Interactions', Category.AEROSOL_CLOUD_INTERACTIONS),
    'ozone': SpeciesID('Ozone', Category.OZONE),
    # Contrails and precursors
    'NOx_aviation': SpeciesID('NOx Aviation', Category.NOX_AVIATION),
    'contrails': SpeciesID('Contrails', Category.CONTRAILS),
    # other minor anthropogenic
    'LAPSI': SpeciesID('Light absorbing particles on snow and ice', Category.LAPSI),
    'H2O_stratospheric': SpeciesID('H2O Stratospheric', Category.H2O_STRATOSPHERIC),
    'land_use': SpeciesID('Land Use', Category.LAND_USE),
    # natural
    'solar': SpeciesID('Solar', Category.SOLAR),
    'volcanic': SpeciesID('Volcanic', Category.VOLCANIC)
}

In [ ]:
scenarios_to_include=['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
df_forc = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')
df_solar = pd.read_csv('../data/forcing/solar_erf.csv')

In [ ]:
emitted_species = [
    'CO2_FFI', 'CO2_AFOLU', 'CH4', 'N2O',
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18',
    'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 
    'HFC-365mfc', 'HFC-4310mee', 'NOx_aviation']
forced_species = ['solar', 'volcanic']
from_other_species = ['CO2', 'ari', 'aci', 'ozone', 'contrails', 'LAPSI', 'H2O_stratospheric', 'land_use']

species_to_include = emitted_species + forced_species + from_other_species

In [ ]:
scenarios = []
for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(emitted_species):
        species_rcmip_name = species.replace("-", "")
        if species == 'NOx_aviation':
            species_rcmip_name = 'NOx|MAGICC Fossil and Industrial|Aircraft'
        elif species == 'CO2_FFI':
            species_rcmip_name = 'CO2|MAGICC Fossil and Industrial'
        elif species == 'CO2_AFOLU':
            species_rcmip_name = 'CO2|MAGICC AFOLU'
        emis_in = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_emis['Region']=='World'), '1750':'2300'
        ].interpolate(axis=1).values.squeeze()

        # CO2 and N2O units need to behave: TODO, sort this out
        if species in ('CO2_FFI', 'CO2_AFOLU', 'N2O'):
            emis_in = emis_in / 1000
        list_of_species.append(Species(species_ids[species], emissions=emis_in))
        #print(species_rcmip_name, emis_in.shape)
        
    # solar and volcanic forcing still a little hacky
    solar_forcing = np.zeros(551)
    solar_forcing[:550] = df_solar['solar_erf'].loc[1750.5:2300].values
    volcanic_forcing = np.zeros(551)
    volcanic_forcing[:270] = df_forc['volcanic'].values
    volcanic_forcing[269:281] = np.linspace(1, 0, 12) * volcanic_forcing[269]
    list_of_species.append(Species(species_ids['solar'], forcing=solar_forcing))
    list_of_species.append(Species(species_ids['volcanic'], forcing=volcanic_forcing))
    
    # add derived species: at this stage just a declaration that we want them
    for species in from_other_species:
        list_of_species.append(Species(species_ids[species]))
        
    scenarios.append(Scenario(scenario, list_of_species))

In [ ]:
valid_all = np.loadtxt('../data/ar6_ensemble_batches/final.csv', dtype=int)#[:1000]
valid_all

In [ ]:
df_cc=pd.read_csv('../data/parameter_sets/carbon_cycle.csv')
df_cr=pd.read_csv('../data/parameter_sets/climate_response.csv')
df_aci=pd.read_csv('../data/parameter_sets/erfaci.csv')
df_ari=pd.read_csv('../data/parameter_sets/erfari.csv')
df_ozone=pd.read_csv('../data/parameter_sets/ozone.csv')
df_scaling=pd.read_csv('../data/parameter_sets/forcing_scaling.csv')
df_1750co2=pd.read_csv('../data/parameter_sets/co2_concentration_1750.csv')

In [ ]:
species_index = {}
for i in range(len(species_to_include)):
    species_index[species_to_include[i]] = i

In [ ]:
seedgen = 1355763
run_config = RunConfig(ch4_lifetime_method=CH4LifetimeMethod.AERCHEMMIP)

calibrated_f4co2_mean = df_cr['F_4xCO2'].mean()

configs = []
for iconf in valid_all:
    config_name = f"{iconf}"

    seed = seedgen + 399 * iconf
    
    # Climate response configs
    climate_response = ClimateResponse(
        ocean_heat_capacity = df_cr.loc[iconf, 'c1':'c3'].values.squeeze(),
        ocean_heat_transfer = df_cr.loc[iconf, 'kappa1':'kappa3'].values.squeeze(),
        deep_ocean_efficacy = df_cr.loc[iconf, 'epsilon'],
        gamma_autocorrelation = df_cr.loc[iconf, 'gamma'],
        sigma_eta = df_cr.loc[iconf, 'sigma_eta'],
        sigma_xi = df_cr.loc[iconf, 'sigma_xi'],
        stochastic_run = True,
        seed = seed
    )

    # Species configs
    # get defaults
    species_config = [copy.copy(species_config_from_default(species)) for species in species_to_include]

    # CO2 carbon cycle sensitivity
    species_config[54].iirf_0 = df_cc.loc[iconf, 'r0']
    species_config[54].iirf_airborne = df_cc.loc[iconf, 'rA']
    species_config[54].iirf_cumulative = df_cc.loc[iconf, 'rC']
    species_config[54].iirf_temperature = df_cc.loc[iconf, 'rT']
    species_config[54].baseline_concentration = df_1750co2.loc[iconf, 'co2_concentration']

    # aerosol indirect params
    species_config[56].aci_params = {
        "scale": df_aci.loc[iconf, "beta"],
        "Sulfur": df_aci.loc[iconf, "shape_so2"], 
        "BC+OC": df_aci.loc[iconf, "shape_bcoc"]
    }

    # methane lifetime params (not varied, defaults are baked in)
    species_config[2].lifetime = 10.788405534387858
    species_config[2].natural_emissions_adjustment = 19.019783117809567
    species_config[2].soil_lifetime = 185

    # aerosol direct params
    for index in df_ari:
        species_config[species_index[index]].erfari_radiative_efficiency = df_ari.loc[iconf, index]

    # forcing scalings!
    for index in df_scaling:
        if index in ['minorGHG', 'solar_amplitude', 'solar_trend', 'CO2']:
            continue
        species_config[species_index[index]].scale = df_scaling.loc[iconf, index]
    species_config[52].scale = df_scaling.loc[iconf, 'solar_amplitude']
    for idx in range(11, 51):
        species_config[idx].scale = df_scaling.loc[iconf, 'minorGHG']

    # overwrite CO2 forcing scaling with correlated parameter from lambda1
    species_config[54].scale = 1+ 0.561*(calibrated_f4co2_mean - df_cr.loc[iconf,'F_4xCO2'])/calibrated_f4co2_mean

    # ozone scalings
    for index in df_ozone:
        if index=='ODS':
            for idx in range(11, 28):
                species_config[idx].ozone_radiative_efficiency = df_ozone.loc[iconf, 'ODS']
        else:
            species_config[species_index[index]].ozone_radiative_efficiency = df_ozone.loc[iconf, index]

    # volcanic efficacy is something like 0.6
    species_config[53].efficacy = 0.6

    configs.append(Config(config_name, climate_response, species_config))

In [ ]:
fair = FAIR(scenarios, configs, run_config=run_config)
fair.run(progress=True)

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16, 10))

for i in range(8):
    ax[i//4,i%4].fill_between(
        np.arange(1750.5, 2301), 
        np.min(fair.temperature[:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        np.max(fair.temperature[:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1750.5, 2301), 
        np.percentile(fair.temperature[:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 5, axis=1), 
        np.percentile(fair.temperature[:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1750.5, 2301), 
        np.percentile(fair.temperature[:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 16, axis=1), 
        np.percentile(fair.temperature[:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].plot(
        np.arange(1750.5, 2301), 
        np.median(fair.temperature[:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        color='#000000',
    )
    ax[i//4,i%4].set_xlim(1750,2300)
    ax[i//4,i%4].set_ylim(-1, 10)
    ax[i//4,i%4].axhline(0, color='k', ls=":", lw=0.5)
    ax[i//4,i%4].set_title(scenarios_to_include[i])
pl.suptitle('Temperature anomaly')

# Temperature diffs w.r.t. 1995-2014

In [ ]:
print(np.percentile(fair.temperature[271:291, 0, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 0, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[291:311, 0, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 0, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[331:351, 0, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 0, :, 0, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
print(np.percentile(fair.temperature[271:291, 1, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 1, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[291:311, 1, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 1, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[331:351, 1, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 1, :, 0, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
print(np.percentile(fair.temperature[271:291, 2, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 2, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[291:311, 2, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 2, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[331:351, 2, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 2, :, 0, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
print(np.percentile(fair.temperature[271:291, 3, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 3, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[291:311, 3, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 3, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[331:351, 3, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 3, :, 0, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
print(np.percentile(fair.temperature[271:291, 7, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 7, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[291:311, 7, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 7, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[331:351, 7, :, 0, 0].mean(axis=0)-fair.temperature[245:265, 7, :, 0, 0].mean(axis=0), (5, 50, 95)))

# Temperature diffs w.r.t. 1850-1900

In [ ]:
print(np.percentile(fair.temperature[271:291, 0, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 0, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[291:311, 0, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 0, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[331:351, 0, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 0, :, 0, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
print(np.percentile(fair.temperature[271:291, 1, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 1, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[291:311, 1, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 1, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[331:351, 1, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 1, :, 0, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
print(np.percentile(fair.temperature[271:291, 2, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 2, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[291:311, 2, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 2, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[331:351, 2, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 2, :, 0, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
print(np.percentile(fair.temperature[271:291, 3, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 3, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[291:311, 3, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 3, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[331:351, 3, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 3, :, 0, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
print(np.percentile(fair.temperature[271:291, 7, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 7, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[291:311, 7, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 7, :, 0, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(fair.temperature[331:351, 7, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 7, :, 0, 0].mean(axis=0), (5, 50, 95)))

## Today

In [ ]:
print(np.percentile(fair.temperature[245:265, 2, :, 0, 0].mean(axis=0)-fair.temperature[100:151, 2, :, 0, 0].mean(axis=0), (5, 50, 95)))

## Forcing

### Methane: SSP245 in 2019

In [ ]:
np.percentile(fair.forcing_array[269, 2, :, 2, 0], (5, 50, 95))

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        fair.forcing_array[:351, 2, :, 2, 0] +
        fair.forcing_array[:351, 2, :, 3, 0] +
        fair.forcing_array[:351, 2, :, 11:51, 0].sum(axis=2) + 
        fair.forcing_array[:351, 2, :, 54, 0],
        5,
        axis=1
    ),
    np.percentile(
        fair.forcing_array[:351, 2, :, 2, 0] +
        fair.forcing_array[:351, 2, :, 3, 0] +
        fair.forcing_array[:351, 2, :, 11:51, 0].sum(axis=2) + 
        fair.forcing_array[:351, 2, :, 54, 0],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        fair.forcing_array[:351, 2, :, 2, 0] +
        fair.forcing_array[:351, 2, :, 3, 0] +
        fair.forcing_array[:351, 2, :, 11:51, 0].sum(axis=2) + 
        fair.forcing_array[:351, 2, :, 54, 0],
        axis=1
    ), color='k'
)

In [ ]:
species_index

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        fair.forcing_array[:351, 2, :, 55:57, 0].sum(axis=2),
        5,
        axis=1
    ),
    np.percentile(
        fair.forcing_array[:351, 2, :, 55:57, 0].sum(axis=2),
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        fair.forcing_array[:351, 2, :, 55:57, 0].sum(axis=2),
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        fair.forcing_array[:351, 2, :, 52, 0],
        5,
        axis=1
    ),
    np.percentile(
        fair.forcing_array[:351, 2, :, 52, 0],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        fair.forcing_array[:351, 2, :, 52, 0],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        fair.forcing_array[:351, 2, :, 53, 0],
        5,
        axis=1
    ),
    np.percentile(
        fair.forcing_array[:351, 2, :, 53, 0],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        fair.forcing_array[:351, 2, :, 53, 0],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        fair.forcing_array[:351, 2, :, 57, 0],
        5,
        axis=1
    ),
    np.percentile(
        fair.forcing_array[:351, 2, :, 57, 0],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        fair.forcing_array[:351, 2, :, 57, 0],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        fair.forcing_array[:351, 2, :, 58, 0],
        5,
        axis=1
    ),
    np.percentile(
        fair.forcing_array[:351, 2, :, 58, 0],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        fair.forcing_array[:351, 2, :, 58, 0],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        fair.forcing_array[:351, 2, :, 59, 0],
        5,
        axis=1
    ),
    np.percentile(
        fair.forcing_array[:351, 2, :, 59, 0],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        fair.forcing_array[:351, 2, :, 59, 0],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        fair.forcing_array[:351, 2, :, 60, 0],
        5,
        axis=1
    ),
    np.percentile(
        fair.forcing_array[:351, 2, :, 60, 0],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        fair.forcing_array[:351, 2, :, 60, 0],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        fair.forcing_array[:351, 2, :, 61, 0],
        5,
        axis=1
    ),
    np.percentile(
        fair.forcing_array[:351, 2, :, 61, 0],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        fair.forcing_array[:351, 2, :, 61, 0],
        axis=1,
    ), color='k'
)

In [ ]:
pl.plot(
    np.arange(1750.5, 2301), 
    fair.temperature[:, 2, :5, 0, 0]-fair.temperature[100:151, 2, :5, 0, 0].mean(axis=0), 
)

## Save out land use forcing

In [ ]:
fair.forcing_array[:, :, :, 61, 0].shape

In [ ]:
np.save('../data/ar6_ensemble_batches/land-use-erf-from-emissions-driven-ssps.npy', fair.forcing_array[:, :, :, 61, 0])